In [12]:
from langchain_ollama.chat_models import ChatOllama
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_ollama import OllamaEmbeddings

llm = ChatOllama(
    model="llama3.2",
    temperature=0.3,
)

embeddings = OllamaEmbeddings(model="mxbai-embed-large")

llm.invoke("Tell me a joke")

AIMessage(content="Here's one:\n\nWhat do you call a fake noodle?\n\nAn impasta.", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-04-17T13:57:51.638282Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2173031625, 'load_duration': 1753744292, 'prompt_eval_count': 29, 'prompt_eval_duration': 209380916, 'eval_count': 18, 'eval_duration': 208758792, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-77f482d2-a8e6-4774-9fe9-4551ee7017ed-0', usage_metadata={'input_tokens': 29, 'output_tokens': 18, 'total_tokens': 47})

In [1]:
from langchain_google_vertexai import ChatVertexAI
from langchain_google_vertexai import VertexAIEmbeddings
from ai_eval.config import global_config as glob

embedding_model = VertexAIEmbeddings(
    project=glob.GCP_PROJECT, model_name="text-embedding-004"
)

llm = ChatVertexAI(model_name="gemini-2.0-flash-001", temperature=0.2)

In [13]:
import numpy as np

class RAG:
    def __init__(self, model="llama3.2"):
        self.llm = llm
        self.embeddings = embedding_model
        self.doc_embeddings = None
        self.docs = None

    def load_documents(self, documents):
        """Load documents and compute their embeddings."""
        self.docs = documents
        self.doc_embeddings = self.embeddings.embed_documents(documents)

    def get_most_relevant_docs(self, query):
        """Find the most relevant document for a given query."""
        if not self.docs or not self.doc_embeddings:
            raise ValueError("Documents and their embeddings are not loaded.")

        query_embedding = self.embeddings.embed_query(query)
        similarities = [
            np.dot(query_embedding, doc_emb)
            / (np.linalg.norm(query_embedding) * np.linalg.norm(doc_emb))
            for doc_emb in self.doc_embeddings
        ]
        most_relevant_doc_index = np.argmax(similarities)
        return [self.docs[most_relevant_doc_index]]

    def generate_answer(self, query, relevant_doc):
        """Generate an answer for a given query based on the most relevant document."""
        prompt = f"question: {query}\n\nDocuments: {relevant_doc}"
        messages = [
            ("system", "You are a helpful assistant that answers questions based on given documents only."),
            ("human", prompt),
        ]
        ai_msg = self.llm.invoke(messages)
        return ai_msg.content

In [14]:
sample_docs = [
    "Albert Einstein proposed the theory of relativity, which transformed our understanding of time, space, and gravity.",
    "Marie Curie was a physicist and chemist who conducted pioneering research on radioactivity and won two Nobel Prizes.",
    "Isaac Newton formulated the laws of motion and universal gravitation, laying the foundation for classical mechanics.",
    "Charles Darwin introduced the theory of evolution by natural selection in his book 'On the Origin of Species'.",
    "Ada Lovelace is regarded as the first computer programmer for her work on Charles Babbage's early mechanical computer, the Analytical Engine."
]

In [15]:
# Initialize RAG instance
rag = RAG()

# Load documents
rag.load_documents(sample_docs)

# Query and retrieve the most relevant document
query = "Who introduced the theory of relativity?"
relevant_doc = rag.get_most_relevant_docs(query)

# Generate an answer
answer = rag.generate_answer(query, relevant_doc)

print(f"Query: {query}")
print(f"Relevant Document: {relevant_doc}")
print(f"Answer: {answer}")

Query: Who introduced the theory of relativity?
Relevant Document: ['Albert Einstein proposed the theory of relativity, which transformed our understanding of time, space, and gravity.']
Answer: According to the document, Albert Einstein introduced the theory of relativity.


In [16]:
sample_queries = [
    "Who introduced the theory of relativity?",
    "Who was the first computer programmer?",
    "What did Isaac Newton contribute to science?",
    "Who won two Nobel Prizes for research on radioactivity?",
    "What is the theory of evolution by natural selection?"
]

expected_responses = [
    "Albert Einstein proposed the theory of relativity, which transformed our understanding of time, space, and gravity.",
    "Ada Lovelace is regarded as the first computer programmer for her work on Charles Babbage's early mechanical computer, the Analytical Engine.",
    "Isaac Newton formulated the laws of motion and universal gravitation, laying the foundation for classical mechanics.",
    "Marie Curie was a physicist and chemist who conducted pioneering research on radioactivity and won two Nobel Prizes.",
    "Charles Darwin introduced the theory of evolution by natural selection in his book 'On the Origin of Species'."
]

In [17]:
dataset = []

for query,reference in zip(sample_queries,expected_responses):

    relevant_docs = rag.get_most_relevant_docs(query)
    response = rag.generate_answer(query, relevant_docs)
    dataset.append(
        {
            "user_input":query,
            "retrieved_contexts":relevant_docs,
            "response":response,
            "reference":reference
        }
    )

In [18]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_list(dataset)
evaluation_dataset

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=5)

In [ ]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(llm)

from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

result = evaluate(dataset=evaluation_dataset,metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],llm=evaluator_llm)
result

Evaluating:  40%|████      | 6/15 [02:36<07:10, 47.81s/it]Exception raised in Job[1]: TimeoutError()
Exception raised in Job[2]: TimeoutError()
Evaluating:  47%|████▋     | 7/15 [03:00<05:18, 39.87s/it]Exception raised in Job[4]: TimeoutError()
Exception raised in Job[7]: TimeoutError()
Exception raised in Job[8]: TimeoutError()
Exception raised in Job[10]: TimeoutError()
Exception raised in Job[11]: TimeoutError()
Exception raised in Job[13]: TimeoutError()
Exception raised in Job[14]: TimeoutError()
Evaluating: 100%|██████████| 15/15 [03:00<00:00, 12.00s/it]


{'context_recall': 0.9000, 'faithfulness': nan, 'factual_correctness(mode=f1)': 1.0000}

In [ ]:
from ragas.embeddings import LangchainEmbeddingsWrapper

generator_embeddings = LangchainEmbeddingsWrapper(embedding_model)

In [21]:
from langchain.document_loaders import PyPDFLoader

filename = "20240731_Nemetschek SE_Mitarbeiterhandbuch_Employee Handbook.pdf"

loader = PyPDFLoader(f"{glob.DATA_PKG_DIR}/{filename}")

docs = loader.load()

In [22]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/18 [00:00<?, ?it/s]unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'Strin

ValueError: No nodes that satisfied the given filer. Try changing the filter.